In [130]:
from app.loader import add_docs_to_knowledge_base
from app.db import create_vectors_index, count_vectors
from IPython import display

In [2]:
await create_vectors_index()

Index idx:vectors created successfully


In [3]:
await add_docs_to_knowledge_base()

Loading documents


100%|██████████| 1/1 [00:04<00:00,  4.14s/it]


Loaded 1 PDF documents

Splitting documents into chunks
Inception.pdf: 127 chunks

Total chunks: 127
Min chunk size: 351 tokens
Max chunk size: 512 tokens
Average chunk size: 496 tokens

Embedding chunks


100%|██████████| 127/127 [00:02<00:00, 45.94it/s]


Adding chunks to vector DB


In [4]:
total_vectors = await count_vectors()
print(f'Total vectors: {total_vectors}')

Total vectors: 127


In [1]:
import json
from time import time
from uuid import uuid4
from IPython.display import display, JSON
from redis.asyncio import Redis
from redis.commands.search.field import TextField, VectorField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from app.config import settings
from app.db import create_chat_index, create_chat, add_chat_messages, get_chat_messages, get_all_chats

r = Redis(host=settings.REDIS_HOST, port=settings.REDIS_PORT)

In [3]:
await create_chat_index()

Error creating index idx:chat: Index already exists


In [4]:
res = await r.ft('idx:chat').search(Query('*'))
for doc in res.docs:
    display(JSON(json.loads(doc.json)))

<IPython.core.display.JSON object>

In [5]:
chat_id = str(uuid4())[:8]
created = int(time())
chat = await create_chat(chat_id, created)
chat

{'id': '376606d1', 'created': 1722970907, 'messages': []}

In [6]:
messages = [
    {'role': 'user', 'content': 'How can you help me?', 'created': int(time())},
    {'role': 'assistant', 'content': 'I am a helpful assistant', 'created': int(time())}
]
await add_chat_messages(chat_id='787acc90', messages=messages)

In [7]:
await get_chat_messages(chat_id='787acc90', last_n=2)

[{'role': 'user', 'content': 'How can you help me?', 'created': 1722970907},
 {'role': 'assistant',
  'content': 'I am a helpful assistant',
  'created': 1722970907}]

In [4]:
await get_all_chats()

[{'id': '376606d1', 'created': 1722970907, 'messages': []},
 {'id': '787acc90',
  'created': 1722968959,
  'messages': [{'role': 'user',
    'content': 'How are you doing?',
    'created': 1722969739},
   {'role': 'assistant',
    'content': 'I am doing well thank you!',
    'created': 1722969739},
   {'role': 'user', 'content': 'How can you help me?', 'created': 1722970232},
   {'role': 'assistant',
    'content': 'I am a helpful assistant',
    'created': 1722970232},
   {'role': 'user', 'content': 'How can you help me?', 'created': 1722970907},
   {'role': 'assistant',
    'content': 'I am a helpful assistant',
    'created': 1722970907}]}]

In [27]:
import openai
from app.openai import client, chat_stream
from app.tools import QueryKnowledgeBaseTool
from app.prompts import MAIN_SYSTEM_PROMPT

In [28]:
message = None
messages = [
    {'role': 'system', 'content': MAIN_SYSTEM_PROMPT},
    {'role': 'user', 'content': 'What is the most resilient parasite?'}
    # {'role': 'user', 'content': 'Greet me and then answer this question: What is the most resilient parasite?'}
    # {'role': 'user', 'content': 'Hello there'}
]
async with chat_stream(
    messages=messages,
    tools=[openai.pydantic_function_tool(QueryKnowledgeBaseTool)],
    tool_choice='auto'
) as stream:
    async for event in stream:
        if event.type == 'content.delta':
            print(event.delta, flush=True, end='')

    assistant_message = (await stream.get_final_completion()).choices[0].message
    print(assistant_message)

# if assistant_message.tool_calls:
#     display(assistant_message.tool_calls[0].function.parsed_arguments)

ParsedChatCompletionMessage[NoneType](content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ParsedFunctionToolCall(id='call_bMXiDnhAzHOBq4NNhvg9cRvz', function=ParsedFunction(arguments='{"query_input":"What is the most resilient parasite?"}', name='QueryKnowledgeBaseTool', parsed_arguments=QueryKnowledgeBaseTool(query_input='What is the most resilient parasite?')), type='function', index=0)], parsed=None)
